In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD, KernelPCA
from sklearn.model_selection import train_test_split, KFold
from sklearn.cluster import KMeans
from mpl_toolkits import mplot3d

In [99]:
df = pd.read_csv("/content/drive/MyDrive/mouse/nestorawa_forcellcycle_expressionMatrix.txt", delimiter="\t")
df2 = pd.read_csv("/content/drive/MyDrive/mouse/nestorowa_corrected_population_annotation.txt")
class_count = len(np.unique(df2['celltype']))

rows = len(df)
df = df.sample(n=int(rows/3))

scaled_df = pd.DataFrame()
reel_classes = []

class_dict = {}
mapped = []
count = 0

for i in df2['celltype']:
  class_label = i.split()[1]
  if class_label not in class_dict:
    class_dict[class_label] = count
    count += 1

  mapped.append(class_dict[class_label])
  reel_classes.append(class_label)

class_count = len(np.unique(reel_classes))
print(class_count)
print(np.unique(reel_classes))

In [88]:
def scaling(column):
  values = []
  scaled_val = []
  for i in df.iloc[:, column]:
    values.append(i)
  mean_val = np.mean(values)
  std = np.std(values)
  for i in df.iloc[:, column]:
    i = (i-mean_val) / std
    scaled_val.append(i)
  scaled_df[df.columns[column]] = scaled_val

In [87]:
for i in range(df.shape[1]):
  scaling(i)

In [93]:
# MY TOOLS

class reduction:
  
  def __init__(self,
               train_data,
               dimension):
    
    self.train_data = train_data
    self.dimension = dimension

  def pca(self):
    from sklearn.decomposition import PCA

    pca = PCA(n_components=self.dimension)
    reducted_data = pca.fit_transform(self.train_data)

    return reducted_data

  def kernel_pca(self,
                 kernel):
    
    from sklearn.decomposition import KernelPCA

    k_pca = KernelPCA(n_components=self.dimension, kernel=kernel)
    reducted_data = k_pca.fit_transform(self.train_data)

    return reducted_data

  def svd(self):
    from sklearn.decomposition import TruncatedSVD

    svd = TruncatedSVD(n_components=self.dimension)
    reducted_data = svd.fit_transform(self.train_data)

    return reducted_data

  def umap(self,
           neighbors,
           metric):
    
    try:
      import umap
    except ImportError:
      !pip install umap-learn
      import umap

    umap = umap.UMAP(n_neighbors=neighbors, n_components=self.dimension, metric=metric)
    embedding = umap.fit_transform(self.train_data)

    return embedding



class metrics:

  def __init__(self,
               data,
               predicted_labels):
    
    self.data = data
    self.predicted_labels = predicted_labels

  def silhouette(self):
    from sklearn.metrics import silhouette_score

    acc_score = silhouette_score(self.data, self.predicted_labels, metric='euclidean')
    
    return acc_score

  def f1_score(self):
    from sklearn.metrics import f1_score

    acc_score = f1_score(self.data, self.predicted_labels)

    return acc_score

  def roc_auc(self,
              predicted_prob,
              multiclass):
    
    from sklearn.metrics import roc_auc_score

    if multiclass:
      acc_score = roc_auc_score(self.data, predicted_prob, multiclass='ovr')

    else: 
      acc_score = roc_auc_score(self.data, predicted_prob)

    return acc_score

  def simple_acc(self):
    from sklearn.metrics import accuracy_score

    acc_score = accuracy_score(self.data, self.predicted_labels)

    return acc_score

  def confusion_matrix(self):
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(self.data, self.predicted_labels)

    return cm



class algorithms:

  def __init__(self,
               train_data,
               train_labels=None,
               test_data=None,
               test_labels=None):
    
    self.random_state = 42
    self.train_data = train_data
    self.train_labels = train_labels
    self.test_data = test_data
    self.test_labels = test_labels

  def kmeans(self,
             class_count):
    
    model = {}
    
    from sklearn.cluster import KMeans

    kmeans = KMeans(n_clusters=class_count, random_state=self.random_state, n_init=20)
    kmeans.fit(self.train_data)

    acc = metrics(self.train_data, kmeans.labels_)
    acc_score = acc.silhouette()

    model['model'] = kmeans
    model['labels'] = kmeans.labels_
    model['acc'] = acc_score

    return model

  def dbscan(self,
             eps,
             min_samples):
    
    model = {}

    from sklearn.cluster import DBSCAN

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    dbscan.fit(self.train_data)

    acc = metrics(self.train_data, dbscan.labels_)
    acc_score = acc.silhouette()

    model['model'] = dbscan
    model['labels'] = dbscan.labels_
    model['acc'] = acc_score

    return model

  def svm(self,
          kernel,
          c):
    
    model = {}
    
    from sklearn.svm import SVC

    svm = SVC(kernel=kernel, C=c, random_state=self.random_state)
    svm.fit(self.train_data, self.train_labels)
    predictions = svm.predict(self.test_data)

    acc = metrics(self.test_labels, predictions)
    acc_score = acc.simple_acc()

    model['model'] = svm
    model['labels'] = predictions
    model['acc'] = acc_score

    return model

  def dtc(self):

    model = {}

    from sklearn.tree import DecisionTreeClassifier

    dtc = DecisionTreeClassifier()
    dtc.fit(self.train_data, self.train_labels)
    predictions = dtc.predict(self.test_data)

    acc = metrics(self.test_labels, predictions)
    acc_score = acc.simple_acc()

    model['model'] = dtc
    model['labels'] = predictions
    model['acc'] = acc_score

    return model

  def rf(self,
         n_estimators,
         max_depth,
         max_features):
    
    model = {}
    
    from sklearn.ensemble import RandomForestClassifier

    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(self.train_data, self.train_labels)
    predictions = rf.predict(self.test_data)

    acc = metrics(self.test_labels, predictions)
    acc_score = acc.simple_acc()

    model['model'] = rf
    model['labels'] = predictions
    model['acc'] = acc_score

    return model

  def mlp(self,
          layers,
          activation_func,
          loss,
          optimizer,
          metric,
          epochs):
    
    model = {}
    
    try:
      from tensorflow.keras.models import Sequential
      from tensorflow.keras.layers import Dense
    except ImportError:
      !pip install tensorflow
      from tensorflow.keras.models import Sequential
      from tensorflow.keras.layers import Dense

    mlp = Sequential()
    for i in layers:
      mlp.add(Dense(layers[i], input_dim=self.train_data.shape[1], activation=activation_func))

    mlp.add(Dense(1, activation='sigmoid'))
    mlp.compile(loss=loss, optimizer=optimizer, metrics=metric)
    mlp.fit(self.train_data, self.train_labels, epochs=epochs)

    threshold = 0.5
    
    predictions = mlp.predict(self.test_data)
    predictions = np.where(predictions > threshold, 1, 0)
    prediction = np.squeeze(predictions)

    acc = metrics(self.test_labels, predictions)
    acc_score = acc.simple_acc()

    model['model'] = mlp
    model['labels'] = predictions
    model['acc'] = acc_score

    return model

  def pca(self,
          dimension):
    
    pca = reduction(self.train_data, dimension)

    return pca.pca()

  def kernel_pca(self,
                 dimension,
                 kernel):
    
    k_pca = reduction(self.train_data, dimension)

    return k_pca.kernel_pca(kernel)

  def svd(self,
          dimension):
    
    svd = reduction(self.train_data, dimension)

    return svd.svd()

  def umap(self,
           neighbors,
           dimension,
           metric):
    umap = reduction(self.train_data, dimension)
    
    return umap.umap(neighbors, metric)

In [57]:
dims = [3, 5, 10]

PCA_data = []
SVD_data = []

for i in dims:
  reducer = algorithms(scaled_df)
  
  reduced = reducer.kernel_pca(i, 'rbf')
  PCA_data.append(reduced)

  reduced = reducer.svd(i)
  SVD_data.append(reduced)

  divided_PCA = []
  divided_SVD = []

  for i in PCA_data:
    divided_PCA.append(i)

  for i in SVD_data:
    divided_SVD.append(i)

In [98]:
from mpl_toolkits.mplot3d import Axes3D

for i, val in enumerate(divided_PCA):

  kmeans = algorithms(val)
  labels = kmeans.kmeans(class_count)['labels']

  acc = metrics(val, labels)
  score = acc.silhouette()

  plt.scatter(val[:, 0], val[:, 1], c=labels, s=50, cmap='plasma')
  plt.title(f'PCA {str(i)} Score : {score}')
  plt.show()

  fig = plt.figure(figsize=(8, 8)) 
  ax = plt.axes(projection='3d') 
  ax.scatter(val[:, 0], val[:, 1], val[:, 2], c=labels, cmap='plasma') 
  ax.set_title('PCA ' + str(i)) 
  plt.show()

for i, val in enumerate(divided_SVD):

  kmeans = algorithms(val)
  labels = kmeans.kmeans(class_count)['labels']

  acc = metrics(val, labels)
  score = acc.silhouette()

  plt.scatter(val[:, 0], val[:, 1], c=labels)
  plt.title(f'SVD {str(i)} Score : {score}')
  plt.show()
  
  fig = plt.figure(figsize=(8, 8)) 
  ax = plt.axes(projection='3d') 
  ax.scatter(val[:, 0], val[:, 1], val[:, 2], c=labels, cmap='viridis') 
  ax.set_title('SVD ' + str(i)) 
  plt.show()

In [101]:
from mpl_toolkits.mplot3d import Axes3D

reducer = algorithms(scaled_df)
embedding = reducer.umap(2, 2, 'euclidean')

kmeans = algorithms(embedding)
labels = kmeans.kmeans(class_count)['labels']

acc = metrics(embedding, labels)
score = acc.silhouette()
print(score)

plt.scatter(embedding[:, 0], embedding[:, 1], c=labels, cmap='plasma')
plt.title('UMAP ' + str(i))
plt.show()

# fig = plt.figure(figsize=(6, 6))
# ax = plt.axes(projection='3d')
# ax.scatter(embedding[:, 0], embedding[:, 1], embedding[:, 2], c=labels, cmap='plasma')
# ax.set_title('UMAP')
# plt.show()